In [1]:
import numpy as np
import pandas as pd
import warnings
import datetime
import string
import json

pd.options.display.max_colwidth = 100
pd.options.display.max_rows = 100
seed = 55
np.random.seed(seed)

warnings.simplefilter(action="ignore", category=pd.core.common.SettingWithCopyWarning)

Reading in dataframes I have prepared

In [2]:
comments_df = pd.read_csv("../data/comments_exploded.csv", index_col=0)
comments_df.head()

,id,title,comments,post_date_utc,post_year,post_month,post_day
0,vzcwal,the princess,Joey King needs a new agent. She’s proven she has talent but she has so many terrible films on h...,1.657851e+09,2022,7,14
1,vzcwal,the princess,"Silly, but entertaining and non stop action",1.657851e+09,2022,7,14
2,vzcwal,the princess,"The yassification of The Raid\n\nActually, this was fun enough and mad respect to Joey King for ...",1.657851e+09,2022,7,14
3,vzcwal,the princess,"Honestly, this was pretty fun. The plot is nothing special yes.\n\nBut Joey King was actually e...",1.657851e+09,2022,7,14
4,vzcwal,the princess,"Man, I loved this movie. Yeah, it was campy, but whatever. The premise worked for me, I liked th...",1.657851e+09,2022,7,14


In [3]:
movies_df = pd.read_csv("../data/reddit_movies_final.csv", index_col=0)
movies_df.head()

,id,title,post_year,post_month,post_day
0,vzcwal,the princess,2022,7,14
1,vzcw0a,the man from toronto,2022,7,14
2,vzcvsd,the sea beast,2022,7,14
3,vzcvkz,mrs. harris goes to paris,2022,7,14
4,vzcv66,where the crawdads sing,2022,7,14


Reading in IMDb datasets for movie titles and ratings

In [4]:
title_basics_imdb = pd.read_csv(
    "../data/imdb/title.basics.tsv.gz", 
    delimiter="\t", 
    compression="gzip",
    na_values=r"\N",
    low_memory=False
    )
title_basics_imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9237277 entries, 0 to 9237276
Data columns (total 9 columns):
 #   Column          Dtype  
---  ------          -----  
 0   tconst          object 
 1   titleType       object 
 2   primaryTitle    object 
 3   originalTitle   object 
 4   isAdult         float64
 5   startYear       float64
 6   endYear         float64
 7   runtimeMinutes  object 
 8   genres          object 
dtypes: float64(3), object(6)
memory usage: 634.3+ MB


In [5]:
title_basics_imdb

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0.0,1892.0,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0.0,1892.0,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0.0,1893.0,NaN,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
9237272,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0.0,2010.0,NaN,NaN,"Action,Drama,Family"
9237273,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0.0,2010.0,NaN,NaN,"Action,Drama,Family"
9237274,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0.0,2010.0,NaN,NaN,"Action,Drama,Family"
9237275,tt9916856,short,The Wind,The Wind,0.0,2015.0,NaN,27,Short


In [6]:
title_basics_imdb['titleType'].value_counts()

tvEpisode       6964547
short            890210
movie            621752
video            265317
tvSeries         231314
tvMovie          137588
tvMiniSeries      45198
tvSpecial         38401
videoGame         32274
tvShort           10674
tvPilot               2
Name: titleType, dtype: int64

In [7]:
# Only include movie types that I think could be discussed in my dataset from r/movies

filtered_title_basics_imdb = title_basics_imdb[
    (title_basics_imdb['titleType'].isin(['movie', 'tvMovie'])) &
    (title_basics_imdb['isAdult'] == 0) &
    (title_basics_imdb['startYear'] >= 2015)
    ]
filtered_title_basics_imdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163914 entries, 11636 to 9237217
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          163914 non-null  object 
 1   titleType       163914 non-null  object 
 2   primaryTitle    163914 non-null  object 
 3   originalTitle   163914 non-null  object 
 4   isAdult         163914 non-null  float64
 5   startYear       163914 non-null  float64
 6   endYear         0 non-null       float64
 7   runtimeMinutes  118429 non-null  object 
 8   genres          157359 non-null  object 
dtypes: float64(3), object(6)
memory usage: 12.5+ MB


In [8]:
filtered_title_basics_imdb = filtered_title_basics_imdb.drop(columns=['endYear', 'isAdult'])

In [9]:
filtered_title_basics_imdb

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres
11636,tt0011801,movie,Tötet nicht mehr,Tötet nicht mehr,2019.0,NaN,"Action,Crime"
13079,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,2021.0,133,Documentary
61093,tt0062336,movie,The Tango of the Widower and Its Distorting Mirror,El Tango del Viudo y Su Espejo Deformante,2020.0,70,Drama
67639,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,2018.0,122,Drama
81238,tt0083060,movie,The Drive to Win,Sha Ou,2019.0,NaN,"Drama,Sport"
...,...,...,...,...,...,...,...
9237090,tt9916460,tvMovie,Pink Taxi,Pink Taxi,2019.0,NaN,Comedy
9237126,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019.0,123,Drama
9237167,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015.0,57,Documentary
9237199,tt9916692,tvMovie,Teatroteka: Czlowiek bez twarzy,Teatroteka: Czlowiek bez twarzy,2015.0,66,Drama


In [10]:
title_ratings_imdb = pd.read_csv("../data/imdb/title.ratings.tsv.gz", delimiter="\t", compression="gzip")

In [11]:
imdb_data = pd.merge(
    left=filtered_title_basics_imdb, 
    right=title_ratings_imdb, 
    how='inner', 
    on="tconst"
    )

`imdb_data` contains info about movie titles and ratings for almost 80,000 movies since 2015

In [12]:
imdb_data

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,2021.0,133,Documentary,6.4,36
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mirror,El Tango del Viudo y Su Espejo Deformante,2020.0,70,Drama,6.4,161
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,2018.0,122,Drama,6.7,7366
3,tt0083060,movie,The Drive to Win,Sha Ou,2019.0,NaN,"Drama,Sport",6.6,33
4,tt0100275,movie,The Wandering Soap Opera,La Telenovela Errante,2017.0,80,"Comedy,Drama,Fantasy",6.4,333
...,...,...,...,...,...,...,...,...,...
78118,tt9916362,movie,Coven,Akelarre,2020.0,92,"Drama,History",6.4,4872
78119,tt9916428,movie,The Secret of China,Hong xing zhao yao Zhong guo,2019.0,NaN,"Adventure,History,War",3.8,14
78120,tt9916460,tvMovie,Pink Taxi,Pink Taxi,2019.0,NaN,Comedy,9.4,18
78121,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019.0,123,Drama,8.3,6


Making `primaryTitle` lowercase in order to use it as a key to join with the Reddit DF.

In [13]:
primaryTitle_Series = imdb_data['primaryTitle'].copy()

In [14]:
imdb_data['primaryTitle'] = imdb_data['primaryTitle'].str.lower()

In [15]:
imdb_data.dtypes

tconst             object
titleType          object
primaryTitle       object
originalTitle      object
startYear         float64
runtimeMinutes     object
genres             object
averageRating     float64
numVotes            int64
dtype: object

In [16]:
imdb_data['runtimeMinutes'] = imdb_data['runtimeMinutes'].astype(float)
imdb_data['startYear'] = imdb_data['startYear'].astype(int)

In [17]:
imdb_data.isna().sum()

tconst               0
titleType            0
primaryTitle         0
originalTitle        0
startYear            0
runtimeMinutes    7351
genres             867
averageRating        0
numVotes             0
dtype: int64

In [18]:
# For now this is fine

In [19]:
movies_df.head()

,id,title,post_year,post_month,post_day
0,vzcwal,the princess,2022,7,14
1,vzcw0a,the man from toronto,2022,7,14
2,vzcvsd,the sea beast,2022,7,14
3,vzcvkz,mrs. harris goes to paris,2022,7,14
4,vzcv66,where the crawdads sing,2022,7,14


Merging the IMDb dataset with Reddit dataset

In [20]:
reddit_imdb_movies = pd.merge(
    left=imdb_data, 
    right=movies_df, 
    how='inner',
    left_on=['primaryTitle', 'startYear'],
    right_on=['title', 'post_year']
    # left_on='primaryTitle',
    # right_on='title'
    )

In [21]:
reddit_imdb_movies

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,id,title,post_year,post_month,post_day
0,tt0069049,movie,the other side of the wind,The Other Side of the Wind,2018,122.0,Drama,6.7,7366,9t6tc7,the other side of the wind,2018,11,2
1,tt0360556,movie,fahrenheit 451,Fahrenheit 451,2018,100.0,"Drama,Sci-Fi,Thriller",4.9,20429,8kpukl,fahrenheit 451,2018,5,19
2,tt0385887,movie,motherless brooklyn,Motherless Brooklyn,2019,144.0,"Crime,Drama,Mystery",6.8,57144,dps11v,motherless brooklyn,2019,10,31
3,tt0437086,movie,alita: battle angel,Alita: Battle Angel,2019,122.0,"Action,Adventure,Sci-Fi",7.3,265007,aoc6mr,alita: battle angel,2019,2,12
4,tt0448115,movie,shazam!,Shazam!,2019,132.0,"Action,Adventure,Comedy",7.0,329951,b94c5i,shazam!,2019,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
784,tt9779516,movie,i still believe,I Still Believe,2020,116.0,"Biography,Drama,Music",6.4,12058,fht3jn,i still believe,2020,3,12
785,tt9784798,movie,judas and the black messiah,Judas and the Black Messiah,2021,126.0,"Biography,Drama,History",7.4,77737,lj1cfb,judas and the black messiah,2021,2,13
786,tt9812474,movie,lamb,Lamb,2021,106.0,"Drama,Fantasy,Horror",6.3,26411,q3nxg1,lamb,2021,10,7
787,tt9827834,movie,sylvie's love,Sylvie's Love,2020,114.0,"Drama,Music,Romance",6.8,5887,kkbe0l,sylvie's love,2020,12,25


**Filtering out duplicates:**

In general, I can do this by keeping only the duplicate title with the most votes on IMDb.

In [22]:
# There are quite a few duplicates.
# Checking numVotes and the movies' IMDb pages (imdb.com/title/<tconst>), these are almost all coincidences.
# I.e., they are similarly named movies that came out in the same year, 
# but aren't the same major motion picture that was discussed on Reddit.
# A few exceptions:
# - "The Girl on the Train," is a 2016 movie with a 2021 Indian remake, and both were discussed on Reddit
# - "The Promise" is actually a 2016 movie, but Reddit discussed it in 2017, so it got matched with the wrong movie on IMDb.
# Luckily, I can filter out the duplicates easily by dropping the movie that has fewer votes on IMDb.

reddit_imdb_movies[reddit_imdb_movies['title'].duplicated(keep=False)]

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,id,title,post_year,post_month,post_day
26,tt10155932,movie,cinderella,Cinderella,2021,113.0,"Comedy,Family,Fantasy",4.2,41769,plccrz,cinderella,2021,9,9
27,tt8907882,movie,cinderella,Cinderella,2021,122.0,"Drama,Horror",6.0,1721,plccrz,cinderella,2021,9,9
37,tt10333426,movie,bliss,Bliss,2021,103.0,"Drama,Romance,Sci-Fi",5.3,17817,ldxwd6,bliss,2021,2,6
38,tt12092584,movie,bliss,Glück,2021,91.0,"Drama,Romance",5.5,266,ldxwd6,bliss,2021,2,6
39,tt10342730,movie,spiral,Spiral: From the Book of Saw,2021,93.0,"Crime,Horror,Mystery",5.2,52440,nbyiaw,spiral,2021,5,13
40,tt9278312,movie,spiral,Spiral,2021,92.0,Drama,7.1,19,nbyiaw,spiral,2021,5,13
64,tt10832274,movie,swan song,Swan Song,2021,105.0,Drama,7.0,2448,rlvt8t,swan song,2021,12,21
65,tt13207508,movie,swan song,Swan Song,2021,112.0,"Drama,Romance,Sci-Fi",6.8,13848,rlvt8t,swan song,2021,12,21
90,tt11252248,movie,dog,Dog,2022,101.0,"Comedy,Drama",6.5,28949,sv796z,dog,2022,2,17
91,tt19880966,movie,dog,Dog,2022,83.0,"Drama,Thriller",8.2,7,sv796z,dog,2022,2,17


In [23]:
imdb_data[imdb_data['primaryTitle'] == 'the promise']

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
34049,tt4192918,movie,the promise,The Promise,2017,115.0,Comedy,8.8,61
38736,tt4776998,movie,the promise,The Promise,2016,133.0,"Action,Adventure,Drama",6.0,176276
51358,tt6072400,movie,the promise,The Promise,2016,90.0,Thriller,6.2,9
61039,tt7232438,movie,the promise,Puen Tee Raluek,2017,114.0,"Drama,Horror,Thriller",5.9,1176


In [24]:
movies_df[movies_df['title'] == 'the promise']

,id,title,post_year,post_month,post_day
787,66mcf3,the promise,2017,4,20


In [25]:
# Drop erroneous Promise
reddit_imdb_movies.drop(index=[403, 404], inplace=True)

# Identify correct Promise; make a one-row df out of it
promise_reddit = movies_df.loc[787]
promise_imdb = imdb_data.loc[38736]
promise = pd.DataFrame(pd.concat([promise_imdb, promise_reddit])).T
promise.index = [reddit_imdb_movies.index[-1] + 1]

# Add Promise back to reddit/imdb df
reddit_imdb_movies = pd.concat([reddit_imdb_movies, promise]).infer_objects()

In [26]:
# Back to dealing with duplicates

In [27]:
duplicate_titles = reddit_imdb_movies[
    reddit_imdb_movies['title'].duplicated(keep=False)
    ].drop(index=[350, 741])['title'].tolist()
duplicate_titles = list(set(duplicate_titles))

duplicate_indices = reddit_imdb_movies[
    reddit_imdb_movies['title'].duplicated(keep=False)
    ].drop(index=[350, 741]).index.tolist()

In [28]:
duplicate_titles

['love and monsters',
 'the circle',
 'they shall not grow old',
 'val',
 'spiral',
 'swan song',
 'cuties',
 'dog',
 'rocketman',
 'anna',
 'the princess',
 'polar',
 'run',
 'bliss',
 'ava',
 'extinction',
 'stronger',
 'coco',
 'cinderella',
 'robin hood']

In [29]:
reddit_imdb_movies[reddit_imdb_movies['title'].isin(duplicate_titles)]['title'].value_counts()

cinderella                 2
bliss                      2
they shall not grow old    2
cuties                     2
robin hood                 2
polar                      2
stronger                   2
extinction                 2
the circle                 2
coco                       2
rocketman                  2
anna                       2
love and monsters          2
the princess               2
run                        2
val                        2
dog                        2
swan song                  2
spiral                     2
ava                        2
Name: title, dtype: int64

In [30]:
# For loop that gets index of the duplicate movie with fewer votes.
# In every case this is the movie that wasn't discussed on Reddit

In [31]:
idx_duplicates_drop = []

for title in duplicate_titles:
    duo = reddit_imdb_movies[reddit_imdb_movies['title'] == title]
    lower_vote = duo['numVotes'].idxmin()
    idx_duplicates_drop.append(lower_vote)

In [32]:
reddit_imdb_movies = reddit_imdb_movies.drop(index=idx_duplicates_drop)

157 movies didn't merge.

Some movies may have been discussed on Reddit in a different year. Some movies might still be misspelled on Reddit.


In [33]:
# Figure out why 157 movies didn't merge

print("Number of movies that didn't merge: ")
print(len(movies_df) - len(reddit_imdb_movies))

Number of movies that didn't merge: 
157


In [34]:
movies_remaining = movies_df[
    ~movies_df['title'].isin(reddit_imdb_movies['title'])
    ]

movies_remaining

,id,title,post_year,post_month,post_day
3,vzcvkz,mrs. harris goes to paris,2022,7,14
6,vtzd6m,marcel the shell with shoes on,2022,7,7
10,vjd5ge,the black phone,2022,6,23
12,v8wvmk,dashcam,2022,6,9
25,uoh6wm,petite maman,2022,5,12
...,...,...,...,...,...
905,4i8038,green room,2016,5,6
907,4gx3iu,ratchet & clank,2016,4,28
913,4dzk1i,demolition,2016,4,8
915,4ducma,hardcore henry,2016,4,7


I'm removing punctuation from titles and trying to merge again, this time without year as a key.

In [35]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [36]:
# Add en dash and em dash to punctuation

punctuation = string.punctuation + "—" "–"

In [37]:
movies_remaining['stripped_title'] \
    = movies_remaining['title'].apply(
        lambda x: x.translate(str.maketrans('', '', punctuation))
        )

imdb_data['stripped_primaryTitle'] \
    = imdb_data['primaryTitle'].apply(
        lambda x: x.translate(str.maketrans('', '', punctuation))
        )

In [38]:
movies_remaining_merged = pd.merge(
    left=imdb_data, 
    right=movies_remaining, 
    how='inner',
    left_on=['stripped_primaryTitle'],
    right_on=['stripped_title']
    )

In [39]:
movies_remaining_merged.duplicated(subset='stripped_title', keep=False).sum()

56

Dealing with duplicates again.

In [40]:
movies_remaining_merged[movies_remaining_merged.duplicated(subset='stripped_title', keep=False)]

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,stripped_primaryTitle,id,title,post_year,post_month,post_day,stripped_title
1,tt0490215,movie,silence,Silence,2016,161.0,"Drama,History",7.1,112814,silence,5no1s5,silence,2017,1,12,silence
2,tt2401348,movie,silence,Silence,2019,NaN,"Drama,Thriller",2.3,15,silence,5no1s5,silence,2017,1,12,silence
3,tt8282116,movie,silence,Silence,2018,90.0,Drama,5.4,22,silence,5no1s5,silence,2017,1,12,silence
5,tt10075374,movie,the little prince,The Little Prince,2019,93.0,Action,7.0,6,the little prince,4wb5lp,the little prince,2016,8,5,the little prince
6,tt1754656,movie,the little prince,Le Petit Prince,2015,108.0,"Adventure,Animation,Drama",7.7,62170,the little prince,4wb5lp,the little prince,2016,8,5,the little prince
7,tt5317732,movie,the little prince,The Little Prince,2015,49.0,Documentary,8.0,5,the little prince,4wb5lp,the little prince,2016,8,5,the little prince
9,tt10272386,movie,the father,The Father,2020,97.0,"Drama,Mystery",8.2,147170,the father,miz3m9,the father,2021,4,2,the father
10,tt10409554,movie,the father,Bashtata,2019,87.0,"Comedy,Drama",6.4,609,the father,miz3m9,the father,2021,4,2,the father
13,tt10666454,movie,supernova,Supernova,2019,78.0,"Drama,Thriller",6.7,626,supernova,lnwobt,supernova,2021,2,19,supernova
14,tt11169050,movie,supernova,Supernova,2020,95.0,"Drama,Romance",6.8,11557,supernova,lnwobt,supernova,2021,2,19,supernova


In [41]:
# Once again, systematically choosing the movie with the most votes.

In [42]:
duplicate_titles_2 = movies_remaining_merged[
    movies_remaining_merged['stripped_title'].duplicated(keep=False)
    ]['stripped_title'].tolist()
duplicate_titles_2 = list(set(duplicate_titles_2))

duplicate_indices_2 = movies_remaining_merged[
    movies_remaining_merged['stripped_title'].duplicated(keep=False)
    ].index.tolist()

In [43]:
movies_remaining_merged[movies_remaining_merged\
    ['stripped_title'].isin(duplicate_titles_2)]['stripped_title'].value_counts()

cargo                         7
monster                       6
emma                          5
split                         5
revenge                       5
silence                       3
the little prince             3
the cursed                    3
sundown                       3
swallow                       2
dashcam                       2
the great wall                2
supernova                     2
the father                    2
hostiles                      2
waiting for the barbarians    2
worth                         2
Name: stripped_title, dtype: int64

In [44]:
# Again, a for loop that only keeps the movie with the highest number of votes.

In [45]:
idx_duplicates_keep = []

for title in duplicate_titles_2:
    group = movies_remaining_merged[movies_remaining_merged['stripped_title'] == title]
    highest_votes = group['numVotes'].idxmax()
    idx_duplicates_keep.append(highest_votes)

idx_duplicates_drop_2 = [idx for idx in duplicate_indices_2 if idx not in idx_duplicates_keep]

In [46]:
movies_remaining_merged.drop(index=idx_duplicates_drop_2, inplace=True)

In [47]:
last_index = reddit_imdb_movies.index[-1]
movies_remaining_merged.index = np.arange(
    last_index + 1, last_index + 1 + len(movies_remaining_merged)
    )

Adding movies back into the DF.

In [48]:
reddit_imdb_movies = \
    pd.concat(
        [
            reddit_imdb_movies,
            movies_remaining_merged.drop(columns=['stripped_primaryTitle', 'stripped_title'])
        ]
    )

There are still 40 movies that haven't merged from the Reddit DF.

In this case, these movies are all spelled differently from how they're spelled on IMDb.

I.e., I will have to manually correct them.

In [49]:
# Figure what's wrong with the remaining 40 movies that haven't merged

print("Number of movies left to merge: ")
print(len(movies_df) - len(reddit_imdb_movies))

Number of movies left to merge: 
40


In [50]:
movies_still_remaining = movies_df[
    ~movies_df['title'].isin(reddit_imdb_movies['title'])
    ]

movies_still_remaining

,id,title,post_year,post_month,post_day
62,sw0jmn,bigbug,2022,2,18
75,s4a86d,hotel transylvania: transformania,2022,1,14
154,opsga7,snake eyes: g.i. joe origins,2021,7,22
159,olt9gx,fear street part three: 1666,2021,7,16
161,ohaa79,fear street part 2: 1978,2021,7,9
164,ocet2k,summer of soul,2021,7,2
165,ocery5,fear street part 1: 1994,2021,7,2
168,oc1ib5,the boss baby: family business,2021,7,1
177,o2ej5a,the hitman's wife's bodyguard,2021,6,17
195,n1y4xd,tom clancy's without remorse,2021,4,30


In [51]:
convert_titles = \
    {
        "bigbug": "big bug",
        "hotel transylvania: transformania": "hotel transylvania 4: transformania",
        "snake eyes: g.i. joe origins": "snake eyes",
        "fear street part three: 1666": "fear street: part three - 1666",
        "fear street part 2: 1978": "fear street: part two - 1978",
        "summer of soul": "summer of soul (...or, when the revolution could not be televised)",
        "fear street part 1: 1994": "fear street: part one - 1994",
        "the boss baby: family business": "the boss baby 2: family business",
        "the hitman's wife's bodyguard": "hitman's wife's bodyguard",
        "tom clancy's without remorse": "without remorse",
        "demon slayer- kimetsu no yaiba / mugen train": "demon slayer the movie: mugen train",
        "saint maude": "saint maud",
        "malcom and marie": "malcolm & marie",
        "the croods 2": "the croods: a new age",
        "midnight sky": "the midnight sky",
        "the christmas chronicles 2": "the christmas chronicles: part two",
        "jingle jangle": "jingle jangle: a christmas journey",
        "possessor: uncut": "possessor",
        "bill and ted face the music": "bill & ted face the music",
        "new mutants": "the new mutants",
        "trolls world tour": "trolls 2: world tour",
        "the current war": "the current war: director's cut",
        "peanut butter falcon": "the peanut butter falcon",
        "rolling thunder revue: a bob dylan story by martin scorsese": "rolling thunder revue",
        "dark phoenix": "x-men: dark phoenix",
        "the girl in the spider's web: a new dragon tattoo story": "the girl in the spider's web",
        "nutcracker and the four realms": "the nutcracker and the four realms",
        "ant man and the wasp": "ant-man and the wasp",
        "sicario: day of soldado": "sicario: day of the soldado",
        "ocean's 8": "ocean's eight",
        "professor marston and the wonder women": "professor marston & the wonder women",
        "john wick 2": "john wick: chapter 2",
        "the divergent series: allegiant": "allegiant",
        "small axe: education": "education",
        "small axe: alex wheatle": "alex wheatle",
        "small axe: red, white, and blue": "red, white and blue",
        "small axe: lover's rock": "lovers rock",
        "small axe: mangrove": "mangrove"
        }

# Jesus is King is a short documentary about Kanye west; doesn't really fit in the dataset
# Once Upon a Deadpool is just an edited version of Deadpool 2
# The "Small Axe" movies are a series of short films. I will have to add them in from the original IMDb dataset

In [52]:
movies_still_remaining.drop(
    index=[443, 578], inplace=True
    )

In [53]:
movies_still_remaining['title'] = movies_still_remaining['title'].map(convert_titles)

In [54]:
small_axe_tconst = ["tt10551100", "tt10551102", "tt10551108", "tt10551104", "tt10551106"]

In [55]:
small_axe_df = pd.merge(
    left=title_basics_imdb[title_basics_imdb['tconst'].isin(small_axe_tconst)], 
    right=title_ratings_imdb, 
    how='inner', 
    on="tconst"
    ).infer_objects()

small_axe_df['primaryTitle'] = small_axe_df['primaryTitle'].apply(lambda x: x.lower())
small_axe_df.drop(columns=['isAdult', 'endYear'], inplace=True)
small_axe_df['runtimeMinutes'] = small_axe_df['runtimeMinutes'].astype(float)
small_axe_df['startYear'] = small_axe_df['startYear'].astype(int)
small_axe_df['stripped_primaryTitle'] \
    = small_axe_df['primaryTitle'].apply(
        lambda x: x.translate(str.maketrans('', '', punctuation))
        )

In [56]:
small_axe_df

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,stripped_primaryTitle
0,tt10551100,tvEpisode,mangrove,Mangrove,2020,127.0,"Drama,History",8.0,5522,mangrove
1,tt10551102,tvEpisode,lovers rock,Lovers Rock,2020,70.0,"Drama,History",7.5,4912,lovers rock
2,tt10551104,tvEpisode,alex wheatle,Alex Wheatle,2020,66.0,"Drama,History",6.8,2299,alex wheatle
3,tt10551106,tvEpisode,education,Education,2020,63.0,"Drama,History",7.6,2236,education
4,tt10551108,tvEpisode,"red, white and blue","Red, White and Blue",2020,80.0,"Drama,History",7.6,3099,red white and blue


In [57]:
imdb_data = pd.concat([
    imdb_data,
    small_axe_df
])

In [58]:
movies_still_remaining_merged = pd.merge(
    left=imdb_data, 
    right=movies_still_remaining, 
    how='right',
    left_on=['primaryTitle'],
    right_on=['title']
    )

In [59]:
last_index = reddit_imdb_movies.index[-1]
movies_still_remaining_merged.index = np.arange(
    last_index + 1, last_index + 1 + len(movies_still_remaining_merged)
    )

In [60]:
movies_still_remaining_merged.tail(3)

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,stripped_primaryTitle,id,title,post_year,post_month,post_day
942,tt6133130,movie,professor marston & the wonder women,Professor Marston and the Wonder Women,2017,108.0,"Biography,Drama,History",7.0,26478,professor marston the wonder women,769dd8,professor marston & the wonder women,2017,10,13
943,tt4425200,movie,john wick: chapter 2,John Wick: Chapter 2,2017,122.0,"Action,Crime,Thriller",7.4,425523,john wick chapter 2,5t4u0w,john wick: chapter 2,2017,2,9
944,tt3410834,movie,allegiant,Allegiant,2016,120.0,"Action,Adventure,Mystery",5.6,122105,allegiant,4b1jgt,allegiant,2016,3,18


Adding movies back into the DF.

In [61]:
reddit_imdb_movies = \
    pd.concat(
        [
            reddit_imdb_movies,
            movies_still_remaining_merged.drop(columns=['stripped_primaryTitle'])
        ]
    )

In [62]:
# Finished merging Reddit movies with IMDb movies

reddit_imdb_movies

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,id,title,post_year,post_month,post_day
0,tt0069049,movie,the other side of the wind,The Other Side of the Wind,2018,122.0,Drama,6.7,7366,9t6tc7,the other side of the wind,2018,11,2
1,tt0360556,movie,fahrenheit 451,Fahrenheit 451,2018,100.0,"Drama,Sci-Fi,Thriller",4.9,20429,8kpukl,fahrenheit 451,2018,5,19
2,tt0385887,movie,motherless brooklyn,Motherless Brooklyn,2019,144.0,"Crime,Drama,Mystery",6.8,57144,dps11v,motherless brooklyn,2019,10,31
3,tt0437086,movie,alita: battle angel,Alita: Battle Angel,2019,122.0,"Action,Adventure,Sci-Fi",7.3,265007,aoc6mr,alita: battle angel,2019,2,12
4,tt0448115,movie,shazam!,Shazam!,2019,132.0,"Action,Adventure,Comedy",7.0,329951,b94c5i,shazam!,2019,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,tt5052474,movie,sicario: day of the soldado,Sicario: Day of the Soldado,2018,122.0,"Action,Adventure,Crime",7.1,146951,8upf4f,sicario: day of the soldado,2018,6,28
941,tt5164214,movie,ocean's eight,Ocean's Eight,2018,110.0,"Action,Comedy,Crime",6.3,217516,8pg30v,ocean's eight,2018,6,7
942,tt6133130,movie,professor marston & the wonder women,Professor Marston and the Wonder Women,2017,108.0,"Biography,Drama,History",7.0,26478,769dd8,professor marston & the wonder women,2017,10,13
943,tt4425200,movie,john wick: chapter 2,John Wick: Chapter 2,2017,122.0,"Action,Crime,Thriller",7.4,425523,5t4u0w,john wick: chapter 2,2017,2,9


Just for good measure, making sure that `title` and `primaryTitle` are the same.

In [63]:
reddit_imdb_movies['title'] = reddit_imdb_movies['primaryTitle']

Finally, I can merge `comments_df` with `reddit_imdb_movies`. I will only keep columns that I think will be useful going forward.

In [64]:
comments_df = \
    comments_df[
        ~comments_df['title'].isin(['once upon a deadpool', 'jesus is king'])
        ]

In [65]:
comments_df.head()

,id,title,comments,post_date_utc,post_year,post_month,post_day
0,vzcwal,the princess,Joey King needs a new agent. She’s proven she has talent but she has so many terrible films on h...,1.657851e+09,2022,7,14
1,vzcwal,the princess,"Silly, but entertaining and non stop action",1.657851e+09,2022,7,14
2,vzcwal,the princess,"The yassification of The Raid\n\nActually, this was fun enough and mad respect to Joey King for ...",1.657851e+09,2022,7,14
3,vzcwal,the princess,"Honestly, this was pretty fun. The plot is nothing special yes.\n\nBut Joey King was actually e...",1.657851e+09,2022,7,14
4,vzcwal,the princess,"Man, I loved this movie. Yeah, it was campy, but whatever. The premise worked for me, I liked th...",1.657851e+09,2022,7,14


In [66]:
reddit_imdb_movies.head()

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,id,title,post_year,post_month,post_day
0,tt0069049,movie,the other side of the wind,The Other Side of the Wind,2018,122.0,Drama,6.7,7366,9t6tc7,the other side of the wind,2018,11,2
1,tt0360556,movie,fahrenheit 451,Fahrenheit 451,2018,100.0,"Drama,Sci-Fi,Thriller",4.9,20429,8kpukl,fahrenheit 451,2018,5,19
2,tt0385887,movie,motherless brooklyn,Motherless Brooklyn,2019,144.0,"Crime,Drama,Mystery",6.8,57144,dps11v,motherless brooklyn,2019,10,31
3,tt0437086,movie,alita: battle angel,Alita: Battle Angel,2019,122.0,"Action,Adventure,Sci-Fi",7.3,265007,aoc6mr,alita: battle angel,2019,2,12
4,tt0448115,movie,shazam!,Shazam!,2019,132.0,"Action,Adventure,Comedy",7.0,329951,b94c5i,shazam!,2019,4,4


In [67]:
data = pd.merge(
    left=comments_df.drop(columns=['title']),
    right=reddit_imdb_movies.drop(columns=[
        'primaryTitle', 'post_year', 
        'post_month', 'post_day'
        ]),
    on='id', 
    how='left'
    )

Rearranging Columns:

In [68]:
data = data[[
    'id', 'tconst', 'title', 'originalTitle',
    'comments', 'runtimeMinutes', 'startYear', 
    'post_date_utc', 'post_year', 'post_month', 'post_day',
    'genres', 'numVotes', 'averageRating'
]]

In [69]:
data.sample(5)

,id,tconst,title,originalTitle,comments,runtimeMinutes,startYear,post_date_utc,post_year,post_month,post_day,genres,numVotes,averageRating
2408,t0tapr,tt15374070,studio 666,Studio 666,I don’t know what to make of this…..just not a good movie at all.\n\nIs this a cash grab by foo ...,106.0,2022,1.645759e+09,2022,2,24,NaN,7356,5.7
21195,imqeot,tt4566758,mulan,Mulan,"just watched it. as a chinese, i can see why this movie tanked in china. i can look past the cha...",115.0,2020,1.599261e+09,2020,9,4,"Action,Adventure,Drama",147078,5.7
22853,h7anh6,tt9686708,the king of staten island,The King of Staten Island,"In my opinion, that tattoo being shitty and something that Ray would hate was a bad choice. The ...",136.0,2020,1.591981e+09,2020,6,12,"Comedy,Drama",67082,7.1
45340,99thr2,tt1308728,the happytime murders,The Happytime Murders,"Just saw it. It’s actually pretty funny. Yeah it’s kinda juvenile at times, but was a good laugh...",91.0,2018,1.535076e+09,2018,8,23,"Action,Comedy,Crime",28293,5.4
20765,iu19h6,tt7395114,the devil all the time,The Devil All the Time,"Obviously Pattinson and Holland knocked it out of the park, but Harry Melling (Roy) absolutely s...",138.0,2020,1.600280e+09,2020,9,16,"Crime,Drama,Thriller",132902,7.1


Save dataframe

In [70]:
# Run this once

# data.to_csv('../data/data_final.csv', header=True, index=False)